In [3]:
#!/Tsan/bin/python
# -*- coding: utf-8 -*-

In [4]:
# Libraries To Use
from __future__ import division 
import numpy as np
import pandas as pd
import os
from datetime import datetime,time,date,timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector 
from itertools import permutations,product
import scipy as sp
import calendar
from imp import reload

In [41]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

In [5]:
%load_ext line_profiler
%matplotlib inline

In [6]:
# to import .py file from other directory
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
data_directory = u'C:/Users/kamis/Desktop/cta合作需求文档/cta合作需求文档/'
data_name = 'adjustAndResample.csv'

In [8]:
data_df = pd.read_csv(data_directory+data_name,infer_datetime_format=True,parse_dates=[0],index_col=0)
data_df.index.name = 'time'
data_df = data_df.head(500)

In [9]:
data_df['time_stamp'] = [x.strftime("%Y-%m-%d %H:%M:%S") for x in data_df.index]

In [10]:
data_df['new_time_stamp'] = [unicode(x) for x in data_df.index]

In [125]:


#df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

trace = go.Ohlc(#x=data_df.index,
                x = data_df['new_time_stamp'],
                open=data_df['raw_open'],
                high=data_df['raw_high'],
                low=data_df['raw_low'],
                close=data_df['raw_close'])
data = [trace]
plotly.offline.plot(data, filename='simple_candlestick')

'file://C:\\Users\\kamis\\Desktop\\myownlibrary\\tradingProject\\simple_candlestick.html'

In [131]:


df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

trace = go.Candlestick(x=df['Date'],
                open=df['AAPL.Open'],
                high=df['AAPL.High'],
                low=df['AAPL.Low'],
                close=df['AAPL.Close'])
data = [trace]
plotly.offline.plot(data, filename='simple_candlestick')

'file://C:\\Users\\kamis\\Desktop\\myownlibrary\\tradingProject\\simple_candlestick.html'

In [72]:
# intraday(minute candlestick) with no gap
trace = go.Candlestick(open=data_df.raw_open,
                high=data_df.raw_high,
                low=data_df.raw_low,
                close=data_df.raw_close)
layout = go.Layout(
    title='financial data',
    xaxis=go.layout.XAxis(
        title='Time',
        tickvals=list(range(len(data_df.index))),
        ticktext=data_df.index),
    yaxis=go.layout.YAxis(
        title='Price'))

data = go.Data([trace])
fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename='intra_day_K_Line-demo')

'file://C:\\Users\\kamis\\Desktop\\myownlibrary\\tradingProject\\intra_day_K_Line-demo.html'

In [116]:
import plotly
plotly.__version__

'3.6.1'

In [117]:
# 更好的画法

In [118]:
INCREASING_COLOR = '#17BECF'
DECREASING_COLOR = '#7F7F7F'

In [119]:
data = [ dict(
    type = 'candlestick',
    open = data_df.raw_open,
    high = data_df.raw_high,
    low = data_df.raw_low,
    close = data_df.raw_close,
    x = data_df.index,
    yaxis = 'y2',
    name = 'original_K_line',
    increasing = dict( line = dict( color = INCREASING_COLOR ) ),
    decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
) ]

layout=dict()

fig = dict( data=data, layout=layout )

In [120]:
xaxis=go.layout.XAxis(
        title='Time',
        tickvals=list(range(len(data_df.index))),
        ticktext=data_df.index)

In [121]:
# Create the layout object
fig['layout'] = dict()
fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'

# fig['layout']['xaxis'] = dict( rangeselector = dict( visible = True ) )
fig['layout']['xaxis'] = go.layout.XAxis(
        title='Time',
        tickvals=list(range(len(data_df.index))),
        ticktext=data_df.index)
fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = True )
fig['layout']['yaxis2'] = dict( domain = [0.2, 0.8] )
fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )

In [122]:
# Add Range Botton
rangeselector=dict(
    visibe = True,
    x = 0, y = 0.9,
    bgcolor = 'rgba(150, 200, 250, 0.4)',
    font = dict( size = 13 ),
    buttons=list([
        dict(count=1,
             label='reset',
             step='all'),
        dict(count=1,
             label='1yr',
             step='year',
             stepmode='backward'),
        dict(count=3,
            label='3 mo',
            step='month',
            stepmode='backward'),
        dict(count=1,
            label='1 mo',
            step='month',
            stepmode='backward'),
        dict(step='all')
    ]))
    
fig['layout']['xaxis']['rangeselector'] = rangeselector

In [123]:
def movingaverage(interval, window_size=10):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [124]:
mv_y = movingaverage(data_df.raw_close)
mv_x = list(data_df.index)

# Clip the ends
mv_x = mv_x[5:-5]
mv_y = mv_y[5:-5]

fig['data'].append( dict( x=mv_x, y=mv_y, type='scatter', mode='lines', 
                         line = dict( width = 1 ),
                         marker = dict( color = '#E377C2' ),
                         yaxis = 'y2', name='Moving Average' ) )

In [125]:
# Set volume bar chart colors
colors = []

for i in range(len(data_df.raw_close)):
    if i != 0:
        if data_df.raw_close[i] > data_df.raw_close[i-1]:
            colors.append(INCREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)
    else:
        colors.append(DECREASING_COLOR)

In [126]:
# Add volume bar chart
fig['data'].append( dict( x=data_df.index, y=data_df.raw_volume,                         
                         marker=dict( color=colors ),
                         type='bar', yaxis='y', name='Volume' ) )

In [127]:
# Add bollinger bands
def bbands(price, window_size=10, num_of_std=5):
    rolling_mean = price.rolling(window=window_size).mean()
    rolling_std  = price.rolling(window=window_size).std()
    upper_band = rolling_mean + (rolling_std*num_of_std)
    lower_band = rolling_mean - (rolling_std*num_of_std)
    return rolling_mean, upper_band, lower_band

bb_avg, bb_upper, bb_lower = bbands(data_df.raw_close)

fig['data'].append( dict( x=data_df.index, y=bb_upper, type='scatter', yaxis='y2', 
                         line = dict( width = 1 ),
                         marker=dict(color='#ccc'), hoverinfo='none', 
                         legendgroup='Bollinger Bands', name='Bollinger Bands') )

fig['data'].append( dict( x=data_df.index, y=bb_lower, type='scatter', yaxis='y2',
                         line = dict( width = 1 ),
                         marker=dict(color='#ccc'), hoverinfo='none',
                         legendgroup='Bollinger Bands', showlegend=False ) )

In [129]:
# plot
plotly.offline.plot( fig, filename = 'candlestick-test-3', validate = False )

'file://C:\\Users\\kamis\\Desktop\\myownlibrary\\tradingProject\\candlestick-test-3.html'